In [1]:
!pip install datasets
!pip install tokenizers
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 9.4 MB/s eta 0:00:00


In [2]:
import torch.nn as nn
import torch
from config import get_config, get_weights_file_path
from train import get_model, get_ds, run_validation,train_model

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device : {device}')
config = get_config()
config['batch_size'] = 32
config['preload'] = None
config['num_epochs'] = 5

device : cuda


In [4]:
def load_model_and_run_validation(config, model_weights_path, num_examples=1):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

    state = torch.load(model_weights_path)
    model.load_state_dict(state['model_state_dict'])

    run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'],
                   device, lambda msg: print(msg), 0, None, num_examples=num_examples)


In [5]:
train_model(config)


device : cuda


Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source sentence: 309
Max length of target sentence: 274
Epoch 0


Processing epoch  0: 100%|██████████| 910/910 [06:26<00:00,  2.35it/s, loss=6.077]


Epoch 1


...

Processing epoch  3: 100%|██████████| 910/910 [06:23<00:00,  2.37it/s, loss=4.834]


Epoch 4


Processing epoch  4: 100%|██████████| 910/910 [06:23<00:00,  2.37it/s, loss=5.031]


Device: cuda
Max length of source sentence: 309
Max length of target sentence: 274


FileNotFoundError: ignored

In [9]:
model_weights_path = get_weights_file_path(config, "04")
load_model_and_run_validation(config, model_weights_path, num_examples=1)

Device: cuda
Max length of source sentence: 309
Max length of target sentence: 274


KeyError: ignored

In [8]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

for batch in val_dataloader:
    print("Batch keys:", batch.keys())

Max length of source sentence: 309
Max length of target sentence: 274
Batch keys: dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'tgt_text'])
Batch keys: dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'tgt_text'])
Batch keys: dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'tgt_text'])
Batch keys: dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'tgt_text'])
Batch keys: dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'tgt_text'])
Batch keys: dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'tgt_text'])
Batch keys: dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'tgt_text'])
Batch keys: dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'tgt_text'])
Batch keys: dict_keys(['encoder_input', 'd

In [10]:
import os
from train import greedy_decode
import torchmetrics
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    source_texts = []
    expected = []
    predicted = []

    try:
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)

            assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            source_text = batch["label"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)

            print_msg('-'*console_width)
            print_msg(f"Source: {source_text}")
            print_msg(f"Target: {target_text}")
            print_msg(f"Predicted: {model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break
    if writer:
        # Compute the character error rate
        metric = torchmetrics.CharErrorrate()
        cer = metric(predicted, expected)
        writer.add_scalar('validation/cer', cer, global_step)
        writer.flush()

        # Compute the word error rate
        metric = torchmetrics.WordErrorRate()
        wer = metric(predicted, expected)
        writer.add_scalar('validation/wer', wer, global_step)
        writer.flush()

        # Compute the BLEU metric
        metric = torchmetrics.BLEUScore()
        bleu = metric(predicted, expected)
        writer.add_scalar('validation/bleu', bleu, global_step)
        writer.flush()

In [12]:
load_model_and_run_validation(config, model_weights_path, num_examples=10)

Device: cuda
Max length of source sentence: 309
Max length of target sentence: 274
--------------------------------------------------------------------------------
Source: tensor([   9,  136, 5721, 1838,   17, 1132,    0,    6,   17,   51,    6,   17,
          32, 3068,    5,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,   